In [1]:
from pykeen.pipeline import pipeline
from pykeen.models import TransE
from pykeen.datasets import YAGO310
from pykeen.models import predict
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator
from pykeen.regularizers import LpRegularizer
from torch.optim import Adam
import torch
import pandas as pd
from tqdm import tqdm
import pickle
import json

In [3]:
model = torch.load('/nas/home/gujiashe/trans/2022-04-11-20-07-24_af660c1f-7064-4859-b5a7-11bd1b476116/replicates/replicate-00000/trained_model.pkl')
dataset = YAGO310()

In [18]:
# Predict tails
data = pd.read_csv('YAGO3-10/test.txt', sep="\t", header=None)
data = data[:1000]
df = {}
df["head"] = []
df["relation"] = []
df["pred_tail"] = []
df["true_tail"] = []
df["rank"] = []

for head, relation, tail in tqdm(data.to_numpy()):
    if head not in dataset.training.entity_to_id.keys() or tail not in dataset.training.entity_to_id.keys():
        continue
    predicted_tails_df = predict.get_tail_prediction_df(
        model, head, relation, triples_factory=dataset.training,
    )

    pred_tail_row = predicted_tails_df[:10]
    pred_tails = pred_tail_row["tail_label"].to_numpy()
    true_tail_row = predicted_tails_df[predicted_tails_df["tail_label"] == tail]
    tail_id = true_tail_row["tail_id"].item()
    rank = predicted_tails_df.index.get_loc(tail_id) + 1
    for pred_tail in pred_tails:
        df["head"].append(head)
        df["relation"].append(relation)
        df["pred_tail"].append(pred_tail)
        df["true_tail"].append(tail)
        df["rank"].append(rank)
    # break
df = pd.DataFrame(df)
df


  0%|          | 0/1000 [00:00<?, ?it/s]/nas/home/gujiashe/miniconda3/envs/pykeen/lib/python3.10/site-packages/pykeen/nn/representation.py:375: UserWarning: Directly use Embedding.shape instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.shape instead of num_embeddings.")
100%|██████████| 1000/1000 [03:23<00:00,  4.92it/s]


,head,relation,pred_tail,true_tail,rank
0,Stan_Collymore,playsFor,Leicester_City_F.C.,England_national_football_team,13
1,Stan_Collymore,playsFor,Aston_Villa_F.C.,England_national_football_team,13
2,Stan_Collymore,playsFor,Wolverhampton_Wanderers_F.C.,England_national_football_team,13
3,Stan_Collymore,playsFor,Walsall_F.C.,England_national_football_team,13
4,Stan_Collymore,playsFor,Real_Oviedo,England_national_football_team,13
...,...,...,...,...,...
9955,Carl_Jung,influences,Anton_LaVey,Alan_Watts,292
9956,Carl_Jung,influences,Sue_Monk_Kidd,Alan_Watts,292
9957,Carl_Jung,influences,Terence_McKenna,Alan_Watts,292
9958,Carl_Jung,influences,Mircea_Eliade,Alan_Watts,292


In [19]:
df.to_csv('yago310_top10_predictions.tsv', sep="\t")

In [9]:
p = pd.read_csv('yago310_top10_predictions.tsv', sep="\t")

In [15]:
p["pred_tail"][0]

"['Leicester_City_F.C.' 'Aston_Villa_F.C.' 'Wolverhampton_Wanderers_F.C.'\n 'Walsall_F.C.' 'Real_Oviedo' 'Fulham_F.C.' 'Crystal_Palace_F.C.'\n 'Bradford_City_A.F.C.' 'Liverpool_F.C.' 'Nottingham_Forest_F.C.']"

In [ ]:
# # Define evaluator
# evaluator = RankBasedEvaluator(
#     filtered=True,  # Note: this is True by default; we're just being explicit
# )

# # Evaluate your model with not only testing triples,
# # but also filter on validation triples
# results = evaluator.evaluate(
#     model=model,
#     mapped_triples=dataset.testing.mapped_triples,
#     additional_filter_triples=[
#         dataset.training.mapped_triples,
#         dataset.validation.mapped_triples,
#     ],
#     device = "cuda:1",
# )

In [89]:
# dataset = YAGO310()
# dataset.relation_to_id.items()

# dataset = YAGO310()

# model = TransE(triples_factory=dataset.training)
# model.to("cuda")

# optimizer = Adam(params=model.get_grad_params()) 

# training_loop = SLCWATrainingLoop(model=model, triples_factory=dataset.training, optimizer=optimizer) 

# training_loop.train(triples_factory=dataset.training, num_epochs=5, batch_size=256)

# evaluator = RankBasedEvaluator()
# # Get triples to test
# mapped_triples = dataset.testing.mapped_triples
# # Evaluate
# results = evaluator.evaluate(model, mapped_triples, batch_size=64)